In [1]:
import pandas as pd
df = pd.read_csv('result/camel_method_prob2.csv', header=0)
df.head()

,commit_id,snippet,is_defect,kinds,snippet_vec,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,knn_proba
0,49f2bb51710a17e1506262d9d58ca1543a747156,@Override\n public Consumer createConsumer(...,True,test,"[-0.007172710262238979, -0.07275532186031342, ...",0.387762,0.229564,0.336521,0.570953,0.497208,0.438477,0.467284,0.428681,0.724114,0.356036,0.594936,0.617135,0.541445,1.0
1,49f2bb51710a17e1506262d9d58ca1543a747156,@Override\n public Producer createProducer(...,True,test,"[-0.015383551828563213, -0.1089896485209465, -...",0.387762,0.229564,0.336521,0.570953,0.497208,0.438477,0.467284,0.428681,0.724114,0.356036,0.594936,0.617135,0.541445,1.0
2,49f2bb51710a17e1506262d9d58ca1543a747156,"public <K, V > BasicCache<K, V> getCache(Strin...",True,test,"[-0.03843029960989952, -0.11351589858531952, -...",0.387762,0.229564,0.336521,0.570953,0.497208,0.438477,0.467284,0.428681,0.724114,0.356036,0.594936,0.617135,0.541445,1.0
3,49f2bb51710a17e1506262d9d58ca1543a747156,@Override\n public void process(Exchange ex...,True,test,"[0.020179931074380875, -0.11824265122413635, -...",0.387762,0.229564,0.336521,0.570953,0.497208,0.438477,0.467284,0.428681,0.724114,0.356036,0.594936,0.617135,0.541445,1.0
4,49f2bb51710a17e1506262d9d58ca1543a747156,public static Properties loadProperties(CamelC...,True,test,"[-0.07928964495658875, -0.04180598258972168, -...",0.387762,0.229564,0.336521,0.570953,0.497208,0.438477,0.467284,0.428681,0.724114,0.356036,0.594936,0.617135,0.541445,1.0


In [2]:
df2 = df[['commit_id', 'is_defect', 'knn_proba']]
df2.head()

,commit_id,is_defect,knn_proba
0,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0
1,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0
2,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0
3,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0
4,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0


In [4]:
df2['non_defect_proba'] = 1 - df2['knn_proba']
df2.head()

C:\Users\masak\AppData\Local\Temp\ipykernel_3836\2394049306.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['non_defect_proba'] = 1 - df2['knn_proba']


,commit_id,is_defect,knn_proba,non_defect_proba
0,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0,0.0
1,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0,0.0
2,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0,0.0
3,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0,0.0
4,49f2bb51710a17e1506262d9d58ca1543a747156,True,1.0,0.0


In [14]:
result = df2.groupby(['commit_id','is_defect'])['non_defect_proba'].prod()
result.head()

commit_id                                 is_defect
000e09a80874cc6b3ee748504611d4bb45be3483  True         0.00
005b2c9a78a357d76d9396b379825d3c033dfac7  True         0.00
005bd727589eb4a670afbe620df63d47bd59f952  False        1.00
00820dbde4688da47e3e75905837db93dd548f7f  False        0.05
008b1ac89b903e1d3c4eed8165faa81606629e92  False        0.00
Name: non_defect_proba, dtype: float64

In [15]:
result2 = result.reset_index()

In [16]:
result2['defect_proba'] = 1 - result2['non_defect_proba']

In [17]:
result2.head()

,commit_id,is_defect,non_defect_proba,defect_proba
0,000e09a80874cc6b3ee748504611d4bb45be3483,True,0.00,1.00
1,005b2c9a78a357d76d9396b379825d3c033dfac7,True,0.00,1.00
2,005bd727589eb4a670afbe620df63d47bd59f952,False,1.00,0.00
3,00820dbde4688da47e3e75905837db93dd548f7f,False,0.05,0.95
4,008b1ac89b903e1d3c4eed8165faa81606629e92,False,0.00,1.00


In [9]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(result2['is_defect'], result2['defect_proba'])

print(f"AUC: {auc}")

AUC: 0.6737221283103919


In [10]:
from sklearn import metrics

result2['pred'] = result2['defect_proba'].apply(lambda x: True if x >= 0.5 else False)
result2.head()

,commit_id,is_defect,non_defect_proba,defect_proba,pred
0,000e09a80874cc6b3ee748504611d4bb45be3483,True,0.00,1.00,True
1,005b2c9a78a357d76d9396b379825d3c033dfac7,True,0.00,1.00,True
2,005bd727589eb4a670afbe620df63d47bd59f952,False,1.00,0.00,False
3,00820dbde4688da47e3e75905837db93dd548f7f,False,0.05,0.95,True
4,008b1ac89b903e1d3c4eed8165faa81606629e92,False,0.00,1.00,True


In [11]:
# クラス予測に基づく評価指標
print("Accuracy:", metrics.accuracy_score(result2['is_defect'], result2['pred']))
# クラス1を正例とした場合
print("クラス1を正例とした場合:")
print("Precision:", metrics.precision_score(result2['is_defect'], result2['pred']))
print("Recall:", metrics.recall_score(result2['is_defect'], result2['pred']))
print("F1 Score:", metrics.f1_score(result2['is_defect'], result2['pred']))

Accuracy: 0.5988023952095808
クラス1を正例とした場合:
Precision: 0.4095878889823381
Recall: 0.7423780487804879
F1 Score: 0.5279132791327913


In [13]:
#         Predicted
#          0    1
#Actual 0  TN   FP
#      1  FN   TP
metrics.confusion_matrix(result2['is_defect'], result2['pred'])

array([[813, 702],
       [169, 487]], dtype=int64)